In [81]:
import pandas as pd
import osmnx as ox
import numpy as np
from networkx import NetworkXNoPath

# Normalizing Citi bike data

The goal here is to load all of the Citi bike trip info and normalize it, joining it to the same rough data structure as we have for the crash data and then have a row for each node that each trip passed through.

First, let's load all of the data from 2019, the last full "normal" year:

In [22]:
from pathlib import Path
from zipfile import ZipFile

trip_df = pd.DataFrame()
data_path = Path('../../data/citibike/')
for zipfile in data_path.glob('2019*.csv.zip'):
    with ZipFile(data_path / zipfile) as zf:
        for file in zf.infolist():
            if file.filename.endswith('.csv') and not file.filename.startswith('__'):
                print(file.filename)
                trip_df = pd.concat((trip_df, pd.read_csv(zf.open(file.filename))))
            
trip_df

201910-citibike-tripdata.csv
201905-citibike-tripdata.csv
201902-citibike-tripdata.csv
201912-citibike-tripdata.csv
201907-citibike-tripdata.csv
201908-citibike-tripdata.csv
201901-citibike-tripdata.csv
201909-citibike-tripdata.csv
201906-citibike-tripdata.csv
201903-citibike-tripdata.csv
201904-citibike-tripdata.csv
201911-citibike-tripdata.csv


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,527,2019-10-01 00:00:05.6180,2019-10-01 00:08:52.9430,3746.0,6 Ave & Broome St,40.724308,-74.004730,223.0,W 13 St & 7 Ave,40.737815,-73.999947,41750,Subscriber,1993,1
1,174,2019-10-01 00:00:15.8750,2019-10-01 00:03:10.1680,3301.0,Columbus Ave & W 95 St,40.791956,-73.968087,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,18264,Subscriber,1992,1
2,759,2019-10-01 00:00:19.8240,2019-10-01 00:12:59.7070,161.0,LaGuardia Pl & W 3 St,40.729170,-73.998102,174.0,E 25 St & 1 Ave,40.738177,-73.977387,25525,Subscriber,1995,1
3,615,2019-10-01 00:00:21.0680,2019-10-01 00:10:36.6790,254.0,W 11 St & 6 Ave,40.735324,-73.998004,477.0,W 41 St & 8 Ave,40.756405,-73.990026,30186,Subscriber,1992,1
4,761,2019-10-01 00:00:26.3800,2019-10-01 00:13:08.3130,161.0,LaGuardia Pl & W 3 St,40.729170,-73.998102,174.0,E 25 St & 1 Ave,40.738177,-73.977387,25597,Subscriber,1992,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478703,534,2019-11-30 23:59:07.8070,2019-12-01 00:08:01.9840,251.0,Mott St & Prince St,40.723180,-73.994800,302.0,Avenue D & E 3 St,40.720828,-73.977932,33020,Subscriber,1996,2
1478704,269,2019-11-30 23:59:14.4120,2019-12-01 00:03:43.6630,3534.0,Frederick Douglass Blvd & W 117 St,40.805159,-73.954692,3509.0,Lenox Ave & W 115 St,40.801194,-73.950074,35136,Subscriber,1974,1
1478705,668,2019-11-30 23:59:26.4970,2019-12-01 00:10:35.2200,408.0,Market St & Cherry St,40.710762,-73.994004,341.0,Stanton St & Mangin St,40.717821,-73.976289,15990,Subscriber,1998,1
1478706,450,2019-11-30 23:59:28.5170,2019-12-01 00:06:58.7670,523.0,W 38 St & 8 Ave,40.754666,-73.991382,519.0,Pershing Square North,40.751873,-73.977706,41169,Subscriber,1996,1


In [86]:
trip_df = trip_df.reset_index()

**NB: The data involved here is too large for us to use Pandas natively. We could certainly use something like Dask but for this first milestone that's a no-go. This means that the trip data we have is perhaps pretty spottily comparable to the crash data that we have.**

For each start and end point in the citi bike trips, we should do the following steps:
1. Compute if the trip is part of our graph. The most correct way to do this would be to compute the shortest path and see if any of its nodes go through our subgraph, but this will take too much time. What we'll do instead is just see if either the start or end of the trip are in our graph and discard the trip if the answer is no.
1. Next, we'll compute the shortest path between the start and end points using our graph and get the node IDs of each node along that path. We'll duplicate the information in the trip row for each node but supply that specific node's info.

We can see that there are only 974 unique stations total in this trip set. So what we can do is quickly calculate which of these stations are in the graph and then categorically filter out all of the station IDs that are not in that set.

In [87]:
pd.concat((trip_df['start station id'], trip_df['end station id'])).unique().shape

(974,)

In [88]:
# Get a dataframe of all of the start station info
start_stations = trip_df[['start station id', 'start station latitude', 'start station longitude']].groupby('start station id').first()
start_stations.columns = ['station_latitude', 'station_longitude']

# Get a dataframe of all of the end station info
end_stations = trip_df[['end station id', 'end station latitude', 'end station longitude']].groupby('end station id').first()
end_stations.columns = ['station_latitude', 'station_longitude']

# Merge the two lists
all_stations = pd.concat((start_stations, end_stations)).drop_duplicates()

all_stations

,station_latitude,station_longitude
72.0,40.767272,-73.993929
79.0,40.719116,-74.006667
82.0,40.711174,-74.000165
83.0,40.683826,-73.976323
116.0,40.741776,-74.001497
...,...,...
3638.0,40.724294,-74.035483
3639.0,40.719252,-74.034234
3640.0,40.733670,-74.062500
3681.0,40.715178,-74.037683


Let's load our graph:

In [89]:
G = ox.graph_from_address("420 East 73rd Street, New York, New York 10021", network_type="bike")

/usr/local/Caskroom/miniconda/base/envs/spokenv/lib/python3.8/site-packages/osmnx/utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/usr/local/Caskroom/miniconda/base/envs/spokenv/lib/python3.8/site-packages/osmnx/utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
/usr/local/Caskroom/miniconda/base/envs/spokenv/lib/python3.8/site-packages/osmnx/utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


Now we filter out all of the stations that aren't in our graph:

In [90]:
# Now we filter out all of the stations that aren't in our graph

# This is the distance from a trip start or endpoint to its nearest node beyond which we assume
# that the crash occurred outside of our graph and isn't really matched to that node.
THRESHOLD_DIST_M = 100
nearest_nodes = pd.DataFrame(columns=['NODE_ID', 'NODE_DIST_FROM_STATION_M', 'NODE_LATITUDE', 'NODE_LONGITUDE'])

for i, station in tqdm(all_stations.iterrows(), total=all_stations.shape[0]):
    nn_id, dist = ox.distance.nearest_nodes(G, station.station_longitude, station.station_latitude, return_dist=True)
    if dist > THRESHOLD_DIST_M:
        continue
    nn = G.nodes[nn_id]
    nearest_nodes.loc[i] = {
        'NODE_ID': nn_id,
        'NODE_DIST_FROM_STATION_M': dist,
        'NODE_LATITUDE': nn['y'],
        'NODE_LONGITUDE': nn['x'],
    }

stations_with_nodes = pd.concat((all_stations, nearest_nodes), join='inner', axis=1)

stations_with_nodes

  0%|          | 0/973 [00:00<?, ?it/s]

,station_latitude,station_longitude,NODE_ID,NODE_DIST_FROM_STATION_M,NODE_LATITUDE,NODE_LONGITUDE
2022.0,40.759107,-73.959223,42424672,21.772276,40.759300,-73.959266
3131.0,40.767128,-73.962246,42431680,20.128203,40.767188,-73.962472
3134.0,40.763126,-73.965269,42449982,34.321289,40.763432,-73.965214
3135.0,40.771129,-73.957723,42430068,77.955723,40.770762,-73.956934
3139.0,40.771183,-73.964094,596776057,13.804652,40.771155,-73.963935
3140.0,40.771404,-73.953517,6263083521,37.884830,40.771699,-73.953291
3141.0,40.765005,-73.958185,6263083532,36.332032,40.765288,-73.957969
3142.0,40.761227,-73.960940,6263094853,39.710201,40.761538,-73.960707
3143.0,40.776321,-73.964274,42436911,4.631181,40.776305,-73.964224
3144.0,40.776777,-73.959010,5375822000,87.766405,40.775989,-73.958953


Now that we have this, we can go through every trip and quickly remove it if either its start or end are not in the index of our station list:

In [92]:
trip_df_in_area = trip_df[trip_df['end station id'].isin(stations_with_nodes.index) & trip_df['start station id'].isin(stations_with_nodes.index)]

trip_df_in_area

,index,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
54,54,605,2019-10-01 00:07:07.3460,2019-10-01 00:17:13.2870,3146.0,E 81 St & 3 Ave,40.775730,-73.956753,3141.0,1 Ave & E 68 St,40.765005,-73.958185,16936,Subscriber,1962,1
63,63,211,2019-10-01 00:08:08.9760,2019-10-01 00:11:40.1600,3141.0,1 Ave & E 68 St,40.765005,-73.958185,3134.0,3 Ave & E 62 St,40.763126,-73.965269,39877,Subscriber,1981,1
183,183,357,2019-10-01 00:22:49.7960,2019-10-01 00:28:47.2940,3141.0,1 Ave & E 68 St,40.765005,-73.958185,3151.0,E 81 St & York Ave,40.772838,-73.949892,16936,Subscriber,1990,1
218,218,416,2019-10-01 00:26:58.4850,2019-10-01 00:33:54.8020,3141.0,1 Ave & E 68 St,40.765005,-73.958185,3151.0,E 81 St & York Ave,40.772838,-73.949892,17685,Subscriber,1969,1
478,478,389,2019-10-01 01:16:11.2450,2019-10-01 01:22:41.1190,3370.0,E 78 St & 2 Ave,40.772797,-73.955778,3370.0,E 78 St & 2 Ave,40.772797,-73.955778,27978,Subscriber,1994,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20551323,1478334,62,2019-11-30 23:19:05.6390,2019-11-30 23:20:07.9310,3370.0,E 78 St & 2 Ave,40.772797,-73.955778,3140.0,1 Ave & E 78 St,40.771404,-73.953517,29794,Subscriber,1979,1
20551334,1478345,2369,2019-11-30 23:20:08.9240,2019-11-30 23:59:38.7180,3156.0,E 72 St & York Ave,40.766638,-73.953483,3156.0,E 72 St & York Ave,40.766638,-73.953483,31518,Subscriber,1964,1
20551347,1478358,117,2019-11-30 23:21:14.4400,2019-11-30 23:23:11.7820,3140.0,1 Ave & E 78 St,40.771404,-73.953517,3135.0,E 75 St & 3 Ave,40.771129,-73.957723,28130,Subscriber,1979,1
20551415,1478426,277,2019-11-30 23:28:54.5520,2019-11-30 23:33:31.9670,3140.0,1 Ave & E 78 St,40.771404,-73.953517,3150.0,E 85 St & York Ave,40.775369,-73.948034,34181,Subscriber,1979,1


Now we want to get the list of nodes involved in each trip and create a new row for each node involved in the trip:

In [95]:
trip = trip_df_in_area.loc[9463]
stations_with_nodes.loc[trip['start station id']].NODE_ID

42437428

In [98]:
trips_with_nodes = pd.DataFrame()
trips_with_no_paths = []

for i, trip in tqdm(trip_df_in_area.iterrows(), total=trip_df_in_area.shape[0]):
    start_node_id = stations_with_nodes.loc[trip['start station id']].NODE_ID
    end_node_id = stations_with_nodes.loc[trip['end station id']].NODE_ID

    try:
        route = ox.shortest_path(G, start_node_id, end_node_id)
    except NetworkXNoPath:
        trips_with_no_paths.append(i)
        continue

    route_df = pd.DataFrame(
        {
            'TRIP_ID': i,
            'NODE_ID': node_id,
            'NODE_LATITUDE': G.nodes[node_id]['y'],
            'NODE_LONGITUDE': G.nodes[node_id]['x'],
        }
        for node_id in route
    )
    
    trips_with_nodes = pd.concat((trips_with_nodes, route_df))
    
trips_with_nodes

  0%|          | 0/202783 [00:00<?, ?it/s]

,TRIP_ID,NODE_ID,NODE_LATITUDE,NODE_LONGITUDE
0,54,42450025,40.775593,-73.956338
1,54,42439403,40.776235,-73.955870
2,54,42439406,40.775293,-73.953637
3,54,42443029,40.774666,-73.954099
4,54,42438506,40.774021,-73.954570
...,...,...,...,...
5,20551441,42429693,40.767495,-73.959317
6,20551441,7064022528,40.767469,-73.959252
7,20551441,7064022522,40.766840,-73.959718
8,20551441,7064022520,40.766209,-73.960149


So now that we have a node for each trip, what we want to do is join that dataframe against the trip dataframe using the trip ID column against the index to get the trip info for each row.

In [99]:
all_nodes_with_trip_info = pd.merge(trips_with_nodes, trip_df_in_area, left_on='TRIP_ID', right_index=True)

In [102]:
all_nodes_with_trip_info = all_nodes_with_trip_info.drop(columns=['index'])

In [103]:
all_nodes_with_trip_info.to_csv('../../data/trip_data_normalized_2019-01-01_2019-12-31.csv')